# 1. 加载依赖包


In [25]:
import thulac
import os
import re
import pandas as pd
from gensim.models import Word2Vec

# 2. 读取停词表


In [26]:

def load_stop_word():
    stop_word = ['的', '。', '是', ' ', '\n', '日', '月', '.', '%','\u3000']
    stop_word_file = 'c:/data/stopwords'
    stop_word_dirs = os.listdir(stop_word_file)
    for textName in stop_word_dirs:
        filename = stop_word_file + '/' + textName
        with open(filename, 'r', encoding='utf-8') as file_object:
            line = file_object.readline()
            while line:
                stop_word.append(line[0:-1])
                line = file_object.readline()
    return stop_word

stop_word = load_stop_word()
print(len(stop_word))
print(stop_word[0:100])

3908
['的', '。', '是', ' ', '\n', '日', '月', '.', '%', '\u3000', '--', '?', '“', '”', '》', '－－', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', "a's", 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'came', 'can', 'cannot', 'cant', "can't", 'cause', 'causes', 'certain']


# 2.5 分词前修改thulac源码中的bug

对thulac的2个bug进行修改



In [27]:
from contextlib import contextmanager
from io import StringIO
import time


@contextmanager
def add_clock_method_to_time():
    py_gt_3_8 = not hasattr(time, "clock")
    if py_gt_3_8:
        setattr(time, "clock", time.perf_counter)
    try:
        yield
    finally:
        if py_gt_3_8:
            delattr(time, "clock")

# 3. 分词

输入一个文本 对文本中的词语 按照 空格 切分 去掉标点等停词
填充一个全局的字典， 词语 -> 词性


In [28]:
 # 默认模式，分词的同时进行词性标注
thu = thulac.thulac(filt=True, seg_only=False)
word_type = {}
def cut_doc(doc):
    with add_clock_method_to_time():
        words = thu.cut(doc)
    words_in_doc = ""
    for word, wordType in words:
        if word in stop_word: continue
        words_in_doc +=  word + " "
        word_type[word] = wordType
        
    return words_in_doc

words_in_doc = cut_doc("用来控制训练的文章数量")
words_in_doc

word_type

Model loaded succeed


{'控制': 'v', '训练': 'v', '文章': 'n', '数量': 'n'}

# 4. 读取数据

每类新闻中抽取1000个新闻构建 dataframe

In [ ]:
my_df = pd.DataFrame([],index=[],columns=['text','label'])

label_num = {}

# 返回 list[str]
def read_content():
    # doc_data 记录 文章数据
    
    # my_df = pd.DataFrame([],index=[],columns=['text','label'])
    
    DOC_COUNT_MAX = 100
    
    
    news_file = 'c:/data/THUCNews'
    news_file_dirs = os.listdir(news_file)
    print(news_file_dirs)
    
    label_index = -1
    for label in news_file_dirs:
        label_index += 1
        label_num[label] = label_index
        
        file = news_file + '/' + label
        doc_ids = os.listdir(file)
        doc_num = 0
        for doc_name in doc_ids:
            doc_num += 1
            if doc_num >= DOC_COUNT_MAX:
                break
            doc_file_name = file + '/' + doc_name
            doc_id = doc_name[0:-4]
            
            doc_content = ""
            
            with open(doc_file_name, 'r', encoding='utf-8') as file_object:
                line = file_object.readline()
                while line:
                    doc_content += line
                    line = file_object.readline()
                    
            # print(doc_name[0:-4], len(doc_content),label)
            my_df.loc[doc_id] = [cut_doc(doc_content), int(label_index)]
            
           
    return my_df

my_df = read_content()

train_df = my_df
train_df.head()

['体育', '娱乐', '家居', '彩票', '房产', '教育', '时尚', '时政', '星座', '游戏', '社会', '科技', '股票', '财经']


In [ ]:
%matplotlib  inline
train_df = my_df
train_df['text_len'] = train_df['text'].apply(lambda x: len(x.split(' ')))
print(train_df['text_len'].describe())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
_ = plt.hist(train_df['text_len'], bins=200)
plt.xlabel('Text char count')
plt.title("Histogram of char count")

In [ ]:
from collections import Counter
all_lines = ' '.join(list(train_df['text']))
word_count = Counter(all_lines.split(' '))
word_count = sorted(word_count.items(), key=lambda d:d[1], reverse = True)

print(len(word_count))

print(word_count[0])
print(word_count[1])
print(word_count[2])

print(word_count[-1])

In [ ]:
# Count Vectors + RidgeClassifier
print(train_df['label'])
train_df['label'] = train_df['label'].astype('int')
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

all_data_size = 120
train_data_size = 100
test_data_size = all_data_size - train_data_size

print(train_df['text'][:train_data_size])

vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])


clf = RidgeClassifier()
clf.fit(train_test[:train_data_size], train_df['label'].values[:train_data_size])
val_pred = clf.predict(train_test[train_data_size:])

print(f1_score(train_df['label'].values[train_data_size:], val_pred, average='macro'))
# 0.28

# svm
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=30, gamma=0.5, decision_function_shape='ovo')
clf.fit(train_test[:train_data_size], train_df['label'].values[:train_data_size])
val_pred = clf.predict(train_test[train_data_size:])

print(f1_score(train_df['label'].values[train_data_size:], val_pred, average='macro'))


from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors = 13 , weights='distance')
clf.fit(train_test[:train_data_size], train_df['label'].values[:train_data_size])
val_pred = clf.predict(train_test[train_data_size:])

print(f1_score(train_df['label'].values[train_data_size:], val_pred, average='macro'))

In [ ]:
# TF-IDF +  RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score


tfidf = TfidfVectorizer(ngram_range=(1,4), max_features=3000)

train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:train_data_size], train_df['label'].values[:train_data_size])

val_pred = clf.predict(train_test[train_data_size:])

print(f1_score(train_df['label'].values[train_data_size:], val_pred, average='macro'))